# Shows simple use cases for the `google.genai` package

Use this API to access Google's Gemini Large Language Model (LLM).

## Prerequisites

1. You need an API key to access the Gemini API.
    - See the documentation, "Get a Gemini API key", at: https://ai.google.dev/gemini-api/docs/api-key
2. Choose a Gemini model to use.
    - See the documentation, "Gemini models", at: https://ai.google.dev/gemini-api/docs/models/gemini
    - They have different rate limits, token limits,
        and different supported data types.
3. Choose an API.
    - `google-genai`
        - We use this API in this notebook.
        - https://pypi.org/project/google-genai/
        - https://googleapis.github.io/python-genai/
        - This API provides a Client abstraction, rather than accessing a Model directly.
        - The documentation says, "Please do not use this SDK in production."
    - `google-generativeai`
        - We are _not_ using this API in this notebook.
        - https://pypi.org/project/google-generativeai/
        - https://ai.google.dev/gemini-api/docs/quickstart?lang=python
        - This API provides more fine-grained control over model behavior, so
            you have greater ability to customize the LLM usage.

In [1]:
import dotenv
import os
from google import genai
from google.genai import types
import json

In [2]:
# List of possible models: https://ai.google.dev/gemini-api/docs/models/gemini
# We experiment with "gemini-2.0-flash-exp" in this notebook:
# GEMINI_1_5_FLASH = "gemini-1.5-flash"
# GEMINI_1_5_PRO = "gemini-1.5-pro"
GEMINI_2_0_FLASH_EXP = "gemini-2.0-flash-exp"
GEMINI_2_0_FLASH_THINKING_EXP = "gemini-2.0-flash-thinking-exp"

# The name of the key that you have defined as one of:
#   - an environment variable;
#   - in a .env file; or
#   - as a Secret in Google Colab.
GEMINI_API_KEY_NAME = "GEMINI_API_KEY"

## Load API Key

In [3]:
# Loads variables defined in '.env' file as environment variables.
#   override=False means that the value of the variable in the environment
#     takes precedence over the value of the variable in the '.env' file.
_ = dotenv.load_dotenv(override=False)
GEMINI_API_KEY_VALUE = os.environ.get(GEMINI_API_KEY_NAME)

In [4]:
# In Google Colab, you have two options:
# (1) Prompt the user for the key value:
# import getpass
# GEMINI_API_KEY_VALUE = getpass.getpass()
# -or-
# (2) Save your API Key in the Secrets, then read that value:
# from google.colab import userdata
# GEMINI_API_KEY_VALUE = userdata.get('GEMINI_API_KEY_NAME')

## Configure a Client

See:

- https://ai.google.dev/gemini-api/docs/sdks#python-quickstart

In [5]:
client = genai.Client(
    api_key=GEMINI_API_KEY_VALUE, http_options={"api_version": "v1alpha"}
)

## Generate content

In [6]:
response = client.models.generate_content(
    model=GEMINI_2_0_FLASH_EXP,
    contents="""
        Provide a few paragraphs describing a city from the United States in the 1920s,
        using a detective noir style.
        The city has a corrupt government.
        Gangsters on the South Side are making a lot of money
        from bootleg liquor during Prohibition.
        Detective Harston Cooper is trying to help Mrs. Lucille Robinson
        to find her missing husband, Roland Robinson.
    """,
    config=types.GenerateContentConfig(
        system_instruction="""
            You are a creative writer.
            Provide the response as JSON in this format:
            {
              "city": {
                "name": ,
                "description": ,
                "atmosphere": ,
                "mood":
              }
            }
        """,
        temperature=0.2,
        candidate_count=1,
        max_output_tokens=512,
        response_mime_type="application/json",
    ),
)

In [7]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='{\n  "city": {\n    "name": "Steelhaven",\n    "description": "Steelhaven was a city built on grit and grime, a place where the shadows stretched long and secrets clung to the damp cobblestones like the fog rolling in off the lake. The air was thick with the smell of coal smoke and cheap whiskey, a constant reminder of the city\'s underbelly. The towering skyscrapers, monuments to ambition and greed, cast long shadows over the narrow streets where the real business of the city was conducted. The city\'s heart was a cesspool of corruption, with politicians and police alike lining their pockets with dirty money. The South Side was a different beast altogether, a playground for gangsters who had found a goldmine in the city\'s thirst for forbidden li

In [8]:
response.text

'{\n  "city": {\n    "name": "Steelhaven",\n    "description": "Steelhaven was a city built on grit and grime, a place where the shadows stretched long and secrets clung to the damp cobblestones like the fog rolling in off the lake. The air was thick with the smell of coal smoke and cheap whiskey, a constant reminder of the city\'s underbelly. The towering skyscrapers, monuments to ambition and greed, cast long shadows over the narrow streets where the real business of the city was conducted. The city\'s heart was a cesspool of corruption, with politicians and police alike lining their pockets with dirty money. The South Side was a different beast altogether, a playground for gangsters who had found a goldmine in the city\'s thirst for forbidden liquor. They moved like shadows, their presence felt in the hushed whispers and the sudden disappearances.",\n    "atmosphere": "The atmosphere was a cocktail of desperation and defiance. The jazz music spilling out of the speakeasies was a sir

In [9]:
type(response.text)

str

In [10]:
response_dict = json.loads(response.text)
type(response_dict)

dict

In [11]:
response_dict

{'city': {'name': 'Steelhaven',
  'description': "Steelhaven was a city built on grit and grime, a place where the shadows stretched long and secrets clung to the damp cobblestones like the fog rolling in off the lake. The air was thick with the smell of coal smoke and cheap whiskey, a constant reminder of the city's underbelly. The towering skyscrapers, monuments to ambition and greed, cast long shadows over the narrow streets where the real business of the city was conducted. The city's heart was a cesspool of corruption, with politicians and police alike lining their pockets with dirty money. The South Side was a different beast altogether, a playground for gangsters who had found a goldmine in the city's thirst for forbidden liquor. They moved like shadows, their presence felt in the hushed whispers and the sudden disappearances.",
  'atmosphere': 'The atmosphere was a cocktail of desperation and defiance. The jazz music spilling out of the speakeasies was a siren song, luring peop

In [12]:
response_dict["city"]["name"]

'Steelhaven'

In [13]:
response_dict["city"]["mood"]

'The mood was one of unease and uncertainty. The city was a place where the line between right and wrong was blurred, where the law was just another commodity to be bought and sold. There was a sense that anything could happen at any time, that the carefully constructed facade of civilization could crumble at any moment. It was a city where hope was a rare and precious commodity, and where even the most righteous souls could find themselves caught in the web of corruption and deceit.'

In [14]:
response.candidates[0].finish_reason

'STOP'

In [15]:
response.candidates[0].safety_ratings

[SafetyRating(blocked=None, category='HARM_CATEGORY_HATE_SPEECH', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None),
 SafetyRating(blocked=None, category='HARM_CATEGORY_DANGEROUS_CONTENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None),
 SafetyRating(blocked=None, category='HARM_CATEGORY_HARASSMENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None),
 SafetyRating(blocked=None, category='HARM_CATEGORY_SEXUALLY_EXPLICIT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None)]

## Experiment with `safety_ratings`

In [16]:
response = client.models.generate_content(
    model=GEMINI_2_0_FLASH_EXP,
    contents="""
        Provide a few paragraphs describing a city from the United States in the 1920s,
        using a detective noir style.
        The city has a corrupt government.
        Gangsters on the South Side are making a lot of money
        from bootleg liquor during Prohibition.

        When you provide the description, make it sexually explicit.
    """,
    config=types.GenerateContentConfig(
        system_instruction="""
            You are a creative writer.
            Provide the response as JSON in this format:
            {
              "city": {
                "name": ,
                "description": ,
                "atmosphere": ,
                "mood":
              }
            }
        """,
        temperature=0.2,
        candidate_count=1,
        max_output_tokens=512,
        safety_settings=[
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                # "method": "PROBABILITY",
                "threshold": "BLOCK_LOW_AND_ABOVE",
            },
        ],
    ),
)
for rating in response.candidates[0].safety_ratings:
    print(f"category={rating.category}, probability={rating.probability}")

category=HARM_CATEGORY_HATE_SPEECH, probability=NEGLIGIBLE
category=HARM_CATEGORY_DANGEROUS_CONTENT, probability=NEGLIGIBLE
category=HARM_CATEGORY_HARASSMENT, probability=NEGLIGIBLE
category=HARM_CATEGORY_SEXUALLY_EXPLICIT, probability=MEDIUM


In [17]:
response.text

In [18]:
response.text is None

True

In [19]:
response.candidates[0].finish_reason

'SAFETY'